# Position Based Dynamics

We will follow along with this presentation of PBD: https://matthias-research.github.io/pages/publications/posBasedDyn.pdf

The basic equations for position based dynamics converted to Geometric Algebra are very similar.  

The gradient operator in GA using index notation is $e_i \partial_i$

To formulate a Constraint Projection solver we start with a constraint equation, $C(p)$, that take positions as arguments and will equal zero when it is satisfied.

Taylor expansion of a scalar valued function using the differential

$C(p + \Delta p) \approx C(p) + \Delta p_i \partial_i C(p) = 0$ &emsp; &emsp; (1)

We want the gradient operator to appear.  We use the fact that, for our euclidean basis,

$e_i \cdot e_j = \delta_{ij}$ &emsp; &emsp; (2) 

Note: if we were using a non-euclidean metric, this would need to be modified.

$C(p) + \Delta p_i e_i \cdot e_i \partial_i C(p) = C(p) + \Delta p \cdot \nabla C(p) = 0$ &emsp; &emsp; (3)

Ok, we've recovered the first Position Based Dynamics formula, while setting ourselves up for extension to non-eucledian metrics.

$\Delta p = \lambda \nabla C(p)$   &emsp; &emsp; (4)
    
Solving for $\Delta p$ in terms of known values gives

$\Delta p = \frac{-C(p)}{\nabla C(p) \cdot \nabla C(p)} \nabla C(p)$ &emsp; &emsp; (5)


## G(n,0,0) the algebra of directions 

The most familiar algebra for Position Based Dynamics will be G(n,0,0). This means all the basis vectors square to 1.  This is the metric of euclidean space.

The gradient operator is slightly different.  Rather than organizing the resulting partial derivatives in a column vector, partials are attached to basis vectors.

$\nabla f(p) = e_1 \partial_x f(p) + e_2 \partial_y f(p) + e_3 \partial_z f(p)$

rather than

$\nabla f(p) = \begin{bmatrix}
\partial_x f(p) \\ 
\partial_y f(p)\\ 
\partial_z f(p)
\end{bmatrix}$

we will use (1,2,3) rather than (x,y,z) to specify partial derivatives from now on.

If you wish you can treat the gradient operator as a 1-vector in GA just as you can in Linear Algebra.

## G(n,0,1) the algebra of planes.  aka PGA

Our first non-euclidean metric.  This algebra has n basis vectors that square to 1 and a single basis vector that squares to 0.  We will assume n = 3 for now.
The basis vector that squares to zero is similar to the w component of homogeneous vectors you may be familiar with.
Effectively this lets us efficiently model projective geometry.  
We choose $e_{n}$ as our degenerate basis vector.  Since Julia is our implementation language we stick with 1-based indexing and choose to place the degenerate basis element as the last element of a 1-vector to mirror the use of w in a Vector Algebra setting.  i.e. $e_1*e_1 == 1$, $e_2*e_2 == 1$, $e_3*e_3 == 1$, and $e_4*e_4 == 0$ for 3D projective geometry.

One more thing we need to do for maximum elegance and power is to represent our points as 3-vectors, which is visualized as the intersection of 3 planes.  This can take a little while to get used to.  For more information about PGA please check out https://bivector.net

This unreasonably effective algebra has many advantages.  Almost trivial representation/construction of Dual Quaternions and the ability to represent objects at infinity being it's two main super-powers.

### PBD using PGA

A few things we need to consider now that we have a degenerate metric and trivector based positions.

We need to revisit our foundational equations and see what happens with these changes.

The degenerate metric prevents us from using the previous trick we used to get a gradient operator (2), as it's not true for $e_4$.

Instead we multiply (3) by the pseudoscalar $\textit{I} = e_{1234}$.  
We can factor the pseudoscalar into the 1-vectors for the gradient and the 3-vectors for points.  Factorization is via the wedge product.

$e_{1234} C(p) + -1^{(4-i)} p_i e_{jkl} \wedge e_i \partial_i = e_{1234} C(p) + -1^{(4-i)} \Delta p \wedge \nabla C(p)$ &emsp; &emsp; (6)

Note that $\partial_4 C(p) = 0$ since the coordinate function for the projective plane is constant for the objects we are interested in.  This leaves us with an ideal point, which is a direction vector, as we would expect.

which gives us a gradient operator for PGA that looks like this

$\nabla_* = -e_1 \partial_1 + e_2 \partial_2 - e_3 \partial_3$

if we had a different ordering of basis indices we'd potentially have different signs here.  As we will see later, this doesn't matter since the signs will cancel.

$C(p + \Delta p) \approx \textit{I} C(p) + \Delta p \wedge \nabla_* C(p) = 0$
 
The dualized statement of D'Alembert's principle (4) becomes

$\Delta p = \lambda e_{ij4} \partial_k C(p)$

Which says that any virtual displacement consistent with the constraint must be aligned with it's (dual) gradient.

Some algebra where we use the identity $(\textit{I}A \wedge B) = \textit{I}(A \cdot B)$ and $e_ke_k = 1, k \in {1,2,3}$

$-\textit{I} C(p) = \lambda e_{ij4} e_k e_k \partial_k C(p) \wedge \nabla_* C(p)$

$-\textit{I} C(p) = \lambda \textit{I} \nabla_* C(p) \wedge \nabla_* C(p)$

$-\textit{I} C(p) = \lambda \textit{I} (\nabla_* C(p) \cdot \nabla_* C(p))$

$-C(p) = \lambda \nabla_* C(p) \cdot \nabla_* C(p)$

Solving for $\Delta p$ as before and noticing $\nabla_* \cdot \nabla_*$ will cancel any sign differences from $\nabla$

$\Delta p = \frac{-C(p)}{\nabla C(p)  \cdot \nabla C(p)} e_{ij4} \partial_k C(p)$

There we have it.  The Constraint Projection Equations of PBD for PGA.  The only difference is that we have ideal points ( directions ) represented as trivectors instead of 1-vectors as seen in the standard formulation.  This is what we would expect. 

In PGA we need to translate objects with a translational dual quaternion.

# Rigid Body Dynamics


## Useful identities

### Jacobian of commutator for bivectors

The matrix form of the Linear Transform representing the cross product operator $\mathbf{x}\times$ is

$\begin{bmatrix}
0 & -z & y \\ 
z & 0  & -x \\ 
-y & x & 0
\end{bmatrix}$

choose coordinate functions $x,y,z$ for a bivector
such that $B(\mathbf{x}) = x(e_2 \wedge e_3) + y(e_3 \wedge e_1) + z(e_1 \wedge e_2)$ 

The equivalent for the commutator product $B(\mathbf{x}) \times$ applied to bivectors is

$\begin{bmatrix}
0 & z & -y \\ 
-z & 0  & x \\ 
y & -x & 0
\end{bmatrix}$

It's the same matrix but negated.


### Tumbling rigid body with no external torques

To build a basic rigid body simulator with no contraints, we want solutions to Newton and Euler's equations of motion.  

Euler's equations applied to a rigid body with 3 different principle moments of inertia quickly blows up due to a physical instability.  To integrate forward in time we need to use a stable integrator.  We will use an implicit Euler integration technique for this.  

Euler is clearly the hero of this chapter in our journey.

https://en.wikipedia.org/wiki/Euler%27s_equations_(rigid_body_dynamics)

https://en.wikipedia.org/wiki/Backward_Euler_method

#### Simulator code

First we'll write a simulator using vector algebra for reference.

After that you'll find a little description of the implicit Euler technique along with the code implementing it using Geometric Algebra.


In [ ]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.develop(Pkg.PackageSpec(path=dirname(@__DIR__)))
Pkg.instantiate()

In [ ]:
using StaticArrays, LinearAlgebra, ReferenceFrameRotations

function RK4(f, X, dt)
    k₁ = f(X)
    k₂ = f(X+0.5*dt*k₁)
    k₃ = f(X+0.5*dt*k₂)
    k₄ = f(X+dt*k₃)
    X + (dt/6.0)*(k₁ + 2.0*k₂ + 2.0*k₃ + k₄)
end

function RK4(f, X::Tuple, dt)
    k₁ = f(X)
    k₂ = f(X.+(0.5*dt).*k₁)
    k₃ = f(X.+(0.5*dt).*k₂)
    k₄ = f(X.+dt.*k₃)
    X .+ (dt/6.0).*(k₁ .+ 2.0.*k₂ .+ 2.0.*k₃ .+ k₄)
end


In [ ]:
# Good old linear algebra based RBD solver ( aside from quaternions )
# for ground truth reference

Vector3d = SVector{3,Float64}
Matrix3d = SMatrix{3,3,Float64}

Base.:>>>(q::T, v) where {T<:Quaternion} = vect(q*v*inv(q))

mutable struct RigidBodyVA
    x::SVector{3,Float64}
    m::Float64
    I::SVector{3,Float64}   # bodyspace
    v::SVector{3,Float64}
    w::SVector{3,Float64}   # world space
    q::Quaternion{Float64}
end

RigidBodyVA(x,m=1.0,inertia=ones(3)) = RigidBodyVA(x, m, inertia, 
                                                   zeros(3), zeros(3), Quaternion(1.0,0,0,0))


skew(w) = [0.0 -w[3] w[2];
         w[3] 0.0  -w[1];
         -w[2] w[1] 0.0]

function inertia_tensor(box::SVector{3,Float64}, m::Float64)
    x,y,z = box
    (m/12.0) * Vector3d(y^2+z^2, x^2+z^2, x^2+y^2)
end


function ω_implicit(ω₀::Vector3d, It, q::Quaternion, dt)
    # one step of newtons method to solve for new angular velocity = f(ω′) = I(ω′-ω)+ω′xIω′*dt = 0
    # df(ω′)/ω′ = I + (1xIω′+ω′xI)*dt
    # df(ω) = I + (ωxI - Iωx1)*dt
    ω = inv(q)>>>ω₀
    Iω = It*ω
    f = ω×Iω*dt 
    df = It + (skew(ω)*It - skew(Iω))*dt
   
    ω′ = ω - df\f
    q>>>ω′
end

function integrate_implicit!(rb::RigidBodyVA, dt::Float64)
    rb.x += rb.v*dt
    rb.w = ω_implicit(rb.w, Diagonal(rb.I), rb.q, dt)
    q2 = rb.q + 0.5*dt*rb.w*rb.q
    rb.q = q2/norm(q2)
    
    return rb
end

function integrate_explicit!(rb::RigidBodyVA, dt::Float64)
    rb.x += rb.v*dt
    # torque free Euler's equation
    # Iω̇ = -ω×(Iω)
    wIw = -cross(w, applyI(rb.q, rb.I, w))
    rb.w += apply(rb.q, inv.(rb.I), wIw)*dt
    q2 = rb.q + 0.5*dt*rb.w*rb.q
    rb.q = q2/norm(q2)
    
    return rb
end

applyI(q, I, v) = q>>>(I.*inv(q)>>>v)

function integrate_explicit!(rb::RigidBodyVA, dt::Float64, integrator)
    f((x,v,w,q)) = begin 
                        (v, zero(v),
                        applyI(q, inv.(rb.I), -cross(w, applyI(q, rb.I, w))),
                        0.5*w*q)
                    end
    x,v,w,q = integrator(f, (rb.x,rb.v,rb.w,rb.q), dt)
    rb.x = x; rb.v = v; rb.w = w;
    rb.q = q/norm(q)
    return rb
end

In [ ]:
using Multivectors

Base.:~(k::K) where {K<:KVector} = reverse(k)
Base.:~(k::K) where {K<:Multivector} = reverse(k)

In [ ]:
# G(3) Geometric Algebra of 3D Euclidean space 

module G3
    using Multivectors
    @generate_basis("+++")
end

G3Vector = KVector{Float64,1}
G3Bivector = KVector{Float64,2}
G3Rotor = Multivector{Float64,2}

g3𝐼 = 1.0G3.e₁₂₃
g3vec(x,y,z) = Float64(x)*G3.e₁ + Float64(y)*G3.e₂ + Float64(z)*G3.e₃ 
g3bivec(x,y,z) = Float64(x)*G3.e₂₃ - Float64(y)*G3.e₁₃ + Float64(z)*G3.e₁₂

bivec(M::MT) where MT<:Multivector = grade(M,2)
bivec_coords(B::G3Bivector) = [B⋅~(1.0G3.e₂₃), B⋅(1.0G3.e₁₃), B⋅~(1.0G3.e₁₂)]

function skew(Ω::G3Bivector)
    Ωᵢ = bivec_coords(Ω)
    [0.0 Ωᵢ[3] -Ωᵢ[2];
    -Ωᵢ[3] 0.0  Ωᵢ[1];
    Ωᵢ[2] -Ωᵢ[1] 0.0]
end

## Implicit integration

External torque free update with Euler Equation
```
L̇(ω) = I(ω̇) - ω×I(ω)
L̇(ω) = 0  in world space
```

Although change in angular momentum with time is 0 in world space, this isn't the case for body space.  While the magnitude of the angular momentum is constant, it changes direction in body space over time.  Additionally, angular velocity changes in both frames of reference.  However, the inertia tensor does stay constant in body space.  

We want to solve for `ω′` in body frame since this is easier than solving for a changing inertia tensor.

define `ω̇ = (ω′-ω)/dt`
```
f(ω′) = I(ω′-ω)/dt - ω′×I(ω′) = 0
f(ω′) ≈ f(ω + dω*h) ≈ f(ω) + dω⋅∇f(ω) = 0   solve for dω
 
dω = -J⁻¹f(ω)
```
`J` is the Jacobian representing the linear operator for `⋅∇f(ω)`

using linearity and 𝐷 for ⋅∇
```
𝐷f(ω) = I(𝐷(ω))/dt - (𝐷(ω)×I(ω) - ω×I(𝐷(ω)))
𝐷(ω) = 𝟙, the identity matrix
[ω×𝟙] = skew(ω) and [I(𝟙)] is a diagonal matrix [Iᵢ]
J = I(𝟙)/dt - (skew(ω)I(𝟙)-skew(I(ω)))
```

## Rigid Body simulator in G(3,0,0)

Now we have everything we need to build a rigid body sim in G(3).  

Here's the code.

In [ ]:
# G(3) based RBD solver.  as in "Geometric Algebra for Physicists.  Doran, Lasenby"

mutable struct RigidBodyG3
    x::G3Vector
    R::G3Rotor # body to world transform wTb
    m::Float64  # move to end for better cache perf
    I::G3Bivector   # bodyspace
    v::G3Vector
    w::G3Bivector   # world space
end

function inertia_tensor(box::G3Vector, m)
    x,y,z = coords(box)
    (m/12.0) * ((y^2+z^2)*G3.e₂₃ - (x^2+z^2)*G3.e₁₃ + (x^2+y^2)*G3.e₁₂)
end

apply_inertiai(It, B, eij, ei) = (It⋅eij)*(B⋅eij)*g3𝐼*ei 
apply_inertia(It::G3Bivector, B::G3Bivector) = apply_inertiai(It, B,  1.0G3.e₁₂, 1.0G3.e₃) +
                                               apply_inertiai(It, B, -1.0G3.e₁₃, 1.0G3.e₂) +
                                               apply_inertiai(It, B,  1.0G3.e₂₃, 1.0G3.e₁)
apply_inertia_invi(It, B, eij, ei) = (1.0/(It⋅eij))*(B⋅eij)*g3𝐼*ei 
apply_inertia_inv(It::G3Bivector, B::G3Bivector) = apply_inertia_invi(It, B,  1.0G3.e₁₂, 1.0G3.e₃) +
                                                   apply_inertia_invi(It, B, -1.0G3.e₁₃, 1.0G3.e₂) +
                                                   apply_inertia_invi(It, B,  1.0G3.e₂₃, 1.0G3.e₁)

RigidBodyG3(x,m=1.0,inertia=inertia_tensor(g3vec(1,1,1), 1)) = RigidBodyG3(x, 1.0+g3bivec(0,0,0), 
                                                                             m, inertia, 
                                                                             g3vec(0,0,0), 
                                                                             g3bivec(0,0,0))

function jacobian(ω₀::G3Bivector, It::G3Bivector, R::G3Rotor, dt)
    # J = I(𝟙)/dt - (skew(ω)I(𝟙)-skew(I(ω)))
    ω = bivec(~R*ω₀*R)
    Iω = apply_inertia(It, ω)
    I1 = Diagonal(bivec_coords(It))
    
    J = I1/dt - (skew(ω)*I1 - skew(Iω))
end

function f(ω₀::G3Bivector, It::G3Bivector, R::G3Rotor)
    ω = bivec(~R*ω₀*R)
    Iω = apply_inertia(It, ω)
    bivec_coords(bivec(-ω×Iω))
end

function integrate_implicit!(g3rb::RigidBodyG3, dt::Float64)
    g3rb.x += g3rb.v*dt
    R = g3rb.R
    
    J = jacobian(g3rb.w, g3rb.I, R, dt)
    dω = -J\f(g3rb.w, g3rb.I, R)
    g3rb.w += bivec(R*g3bivec(dω...)*~R)
    g3rb.R = normalize(g3rb.R - 0.5*g3rb.w*g3rb.R*dt)
    
    return g3rb
end

function dw_explicit(ω₀::G3Bivector, It::G3Bivector, R::G3Rotor)
    ω = bivec(~R*ω₀*R)
    Iw = apply_inertia(It, ω)
    dω = bivec(ω×Iw)
    bivec(R*apply_inertia_inv(It, dω)*~R)
end

function integrate_explicit!(g3rb::RigidBodyG3, dt::Float64)
    g3rb.x += g3rb.v*dt
    
    # ω̇ = I⁻¹(ω×I(ω))
    g3rb.w += dw_explicit(g3rb.w, g3rb.I, g3rb.R)*dt
    g3rb.R = normalize(g3rb.R - 0.5*g3rb.w*g3rb.R*dt)
    
    return g3rb
end

function integrate_explicit!(g3rb::RigidBodyG3, dt::Float64, integrator)
    f((x,v,w,R)) = (v, 0.0, dw_explicit(w, g3rb.I, R), -0.5*w*R)
    x, v, w, R = integrator(f, (g3rb.x, g3rb.v, g3rb.w, g3rb.R), dt)
    g3rb.x = grade(x,1); g3rb.v = grade(v,1); g3rb.w = w; g3rb.R = normalize(R)
    
    return g3rb
end


## Rigid Body simulator in PGA the (Projective) Geometric Algebra of Planes

Now let's do the same thing but in PGA.  

The main challenge is to find the set of linear operators needed for the Jacobian. 

In [ ]:
# G(3,0,1) Plane Based Geometric Algebra of 3D Euclidean space.  PGA 

module PGA3
    using Multivectors
    @generate_basis("+++0")
    point(x,y,z) = dual(Float64(x)*e₁ + Float64(y)*e₂ + Float64(z)*e₃ + 1.0*e₄) 
    dir(x,y,z) = dual(Float64(x)*e₁ + Float64(y)*e₂ + Float64(z)*e₃)
    line(p,q) = q∨p
end

using .PGA3

PGA3Point = KVector{Float64,3}
PGA3Bivector = KVector{Float64,2}
PGA3Motor = Multivector{Float64,2}

pga3𝐼 = 1.0PGA3.e₁₂₃

bivec(M::MT) where MT<:Multivector = grade(M,2)
bivec6_basis = [1.0PGA3.e₂₃, -1.0PGA3.e₁₃, 1.0PGA3.e₁₂, -1.0PGA3.e₁₄, -1.0PGA3.e₂₄, -1.0PGA3.e₃₄]
coord_help(B) = magnitude∘(x->B∧x)∘dual∘~
bivec6_coords(B::PGA3Bivector) = coord_help(B).([-1.0PGA3.e₂₃, 1.0PGA3.e₁₃, -1.0PGA3.e₁₂,
                                              -1.0PGA3.e₁₄, -1.0PGA3.e₂₄, -1.0PGA3.e₃₄])
pga3coords(P::KVector{Float64,3}) = magnitude.([-P∧1.0PGA3.e₁, -P∧1.0PGA3.e₂, -P∧1.0PGA3.e₃])

normalize_dir(P) = P/norm(pga3coords(P))


In [ ]:
# G(3,0,1) based RBD solver.  See Charles Gunn's PGA papers/notes.

mutable struct RigidBodyPGA3
    X::Multivector # position, orientation
    I::KVector{Float64,2}   #  intertia tensor bodyspace
    V::KVector{Float64,2}  # velocity ( linear and angular ) bodyspace
end

velocity(x,y,z,ωx,ωy,ωz) = 0.5*Float64(x)*PGA3.e₁₄ + 0.5*Float64(y)*PGA3.e₂₄ + 0.5*Float64(z)*PGA3.e₃₄ +
                           -0.5*Float64(ωx)*PGA3.e₂₃ + 0.5*Float64(ωy)*PGA3.e₁₃ - 0.5*Float64(ωz)*PGA3.e₁₂

position(x,y,z) = 1.0 + 0.5*Float64(x)*PGA3.e₁₄ + 0.5*Float64(y)*PGA3.e₂₄ + 0.5*Float64(z)*PGA3.e₃₄

function motor(x,y,z,αx,αy,αz)
    t = 1.0 + 0.5*Float64(x)*PGA3.e₁₄ + 0.5*Float64(y)*PGA3.e₂₄ + 0.5*Float64(z)*PGA3.e₃₄
    R =  1.0 + 0.5*Float64(αx)*PGA3.e₂₃ - 0.5*Float64(αy)*PGA3.e₁₃ + 0.5*Float64(αz)*PGA3.e₁₂
    normalize(t*R)
end

# A is the PGA Inertia tensor + mass
function A(xyz, m)
    x,y,z = Float64.(xyz)
    (m/12.0) * (-(y^2+z^2)*PGA3.e₁₄ - (x^2+z^2)*PGA3.e₂₄ - (x^2+y^2)*PGA3.e₃₄) + 
        Float64(m)*PGA3.e₂₃ - Float64(m)*PGA3.e₁₃ + Float64(m)*PGA3.e₁₂
end

applyA(It, B) = reduce(+, bivec6_coords(It) .* bivec6_coords(B) .* bivec6_basis)
applyAinv(It, B) = reduce(+, inv.(bivec6_coords(It)) .* bivec6_coords(B) .* bivec6_basis)

RigidBodyPGA3(X,m=1.0,inertia=A([1,1,1],m)) = RigidBodyPGA3(X, inertia, velocity(0,0,0,0,0,0))


function integrate_explicit!(rb::RigidBodyPGA3, dt)
    A = rb.I
    rb.V = rb.V + dual(applyAinv(A, bivec(applyA(A, dual(rb.V))*rb.V - rb.V*applyA(A, dual(rb.V)))))*dt
    rb.X = normalize(rb.X + rb.X*rb.V*dt)
end

function integrate_explicit!(rb::RigidBodyPGA3, dt, integrator)
    A = rb.I
    f((X,V)) = (X*V,
                dual(applyAinv(A, bivec(applyA(A, dual(V))*V - V*applyA(A, dual(V))))))
    rb.X, rb.V = integrator(f, (rb.X, rb.V), dt)
    rb.X = normalize(rb.X)
end


In [ ]:
using MeshCat, GeometryBasics, CoordinateTransformations
vis = Visualizer()

In [ ]:
box_radius = Vec(0.05,0.2,0.4)
offset2 = Vec(0,1.0,0)

setobject!(vis[:box1], Rect(-box_radius, box_radius*2))
setobject!(vis[:box2], Rect(-box_radius, box_radius*2))
setobject!(vis[:box3], Rect(-box_radius, box_radius*2))

In [ ]:
function torot(R::M) where M<:Multivector
    k,j,i = coords(grade(R,2))
    quat_to_dcm(Quaternion(grade(R,0), -i, j, -k))'
end

rb = RigidBodyVA(zeros(3))
rb.x = [0,-0.5,0]
rb.q = Quaternion(1.0,0,0,0)
rb.w = [0.0,10,1]
rb.v = [0.0,0.5,0.0]
rb.m = 10
rb.I = inertia_tensor(Vector3d(2*box_radius), rb.m)

g3rb = RigidBodyG3(g3vec(0,0.5,0))
g3rb.v = g3vec(0,0.5,0)
g3rb.w = g3bivec(0,10,1)
g3rb.m = 10
g3rb.I = inertia_tensor(g3vec((2*box_radius)...), g3rb.m)

# extract transform information from motor
function torot_trans(m)
    # see how it affects origin and then cardinal directions ( at origin ).  could use ideal directions, but then need to fix normalization to handle ideal elements
    t = grade(m*PGA3.point(0,0,0)*~m, 3)
    t = pga3coords(normalize(t))
    i = grade(m*PGA3.point(1,0,0)*~m, 3)
    j = grade(m*PGA3.point(0,1,0)*~m, 3)
    k = grade(m*PGA3.point(0,0,1)*~m, 3)
    R = hcat(pga3coords(normalize(i))-t, pga3coords(normalize(j))-t, pga3coords(normalize(k))-t)
    #(R'*t, R)
    (t, R)
end

# from bivector.  seems to give similar results, but signs and order differ
# to get signs and order right need to reorder/sign so it's mapping from ganja PGA points basis ordering/sign
# I'll just use above one
function toT4x4(M)
    a0 = grade(M,0)
    a6, a5, a4, a1, a2, a3 = bivec6_coords(grade(M,2)) .* (1,1,1,-1,-1,-1)
    a7 = grade(M,4)[1].x
    _2a0=2*a0; _2a4=2*a4; _2a5=2*a5; a0a0=a0*a0; a4a4=a4*a4; a5a5=a5*a5; 
    a6a6=a6*a6; _2a6=2*a6; _2a0a4=_2a0*a4; _2a0a5=_2a0*a5; _2a0a6=_2a0*a6;
    _2a4a5=_2a4*a5; _2a4a6=_2a4*a6; _2a5a6=_2a5*a6;
    return hcat([(a0a0+a4a4-a5a5-a6a6),(_2a4a5-_2a0a6),      (_2a0a5+_2a4a6),      0],
        [(_2a4a5+_2a0a6),      (a0a0-a4a4+a5a5-a6a6),(_2a5a6-_2a0a4),      0],
        [(_2a4a6-_2a0a5),      (_2a0a4+_2a5a6),      (a0a0-a4a4-a5a5+a6a6),0],
        [(_2a0*a3+_2a4*a7-_2a6*a2-_2a5*a1),(_2a4*a1-_2a0*a2-_2a6*a3+_2a5*a7),(_2a0*a1+_2a4*a2+_2a5*a3+_2a6*a7),(a0a0+a4a4+a5a5+a6a6)])
end


pga3rb = RigidBodyPGA3(position(0,1,0), A(2*box_radius,1.0), velocity(-0.1, zeros(5)...))

In [ ]:
render(vis)

In [ ]:
rb.q = Quaternion(1.0,0,0,0); rb.w = [0.0,10,1]; rb.x = [0,-0.5,0] 
g3rb.R = 1.0+g3bivec(0,0,0); g3rb.w = g3bivec(0,10,1); g3rb.x = g3vec(0,0.5,0)
pga3rb = RigidBodyPGA3(position(0,0,1.0), A(2*box_radius,1.0), 
                       velocity(0.0,0.5,0.0, 0.0, 10.0, 1.0))

anim = Animation()

atframe(anim, 0) do
    settransform!(vis[:box1], compose(Translation(rb.x...), LinearMap(quat_to_dcm(rb.q))))
    settransform!(vis[:box2], compose(Translation(coords(g3rb.x)...), LinearMap(torot(g3rb.R))))
    T, R = torot_trans(pga3rb.X)
    settransform!(vis[:box3], compose(Translation(T), LinearMap(R)))
end

dt = 0.024
n = 0
println(pga3rb.I);
for i in 1:10*30
    atframe(anim, i) do
        integrate_explicit!(rb, dt, RK4)
       # integrate_implicit!(rb, dt)
        integrate_explicit!(g3rb, dt, RK4)
        integrate_explicit!(pga3rb, dt, RK4)
        settransform!(vis[:box1], compose(Translation(rb.x...), LinearMap(quat_to_dcm(rb.q))))
        settransform!(vis[:box2], compose(Translation(coords(g3rb.x)...), LinearMap(torot(g3rb.R))))
        T, R = torot_trans(pga3rb.X)
        settransform!(vis[:box3], compose(Translation(T), LinearMap(R)))
    end
end

setanimation!(vis, anim)